In [20]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [21]:
train_labels = pd.read_csv('Archivos de datos/train_labels.csv', encoding='latin-1')
train_values = pd.read_csv('Archivos de datos/train_values.csv', encoding='latin-1')

join = train_values.merge(train_labels)

## Experimentacion

join.columns

join['damage_grade'].value_counts()

X=pd.get_dummies(join.loc[:,:'has_secondary_use_other'])
y=join['damage_grade'].astype(int)

## Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=1)

In [22]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

params = {
    'metric': 'rmse',
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'force_row_wise':True,
}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)

preds = gbm.predict(X_test, num_iteration=gbm.best_iteration)

[LightGBM] [Info] Total Bins 1066
[LightGBM] [Info] Number of data points in the train set: 208480, number of used features: 68
[LightGBM] [Info] Start training from score 2.238598
[1]	valid_0's rmse: 0.600618
Training until validation scores don't improve for 10 rounds
[2]	valid_0's rmse: 0.591603
[3]	valid_0's rmse: 0.583362
[4]	valid_0's rmse: 0.575859
[5]	valid_0's rmse: 0.568956
[6]	valid_0's rmse: 0.562583
[7]	valid_0's rmse: 0.556742
[8]	valid_0's rmse: 0.551408
[9]	valid_0's rmse: 0.546516
[10]	valid_0's rmse: 0.543614
[11]	valid_0's rmse: 0.539363
[12]	valid_0's rmse: 0.535454
[13]	valid_0's rmse: 0.533214
[14]	valid_0's rmse: 0.529834
[15]	valid_0's rmse: 0.527913
[16]	valid_0's rmse: 0.524936
[17]	valid_0's rmse: 0.522199
[18]	valid_0's rmse: 0.519667
[19]	valid_0's rmse: 0.517335
[20]	valid_0's rmse: 0.516054
[21]	valid_0's rmse: 0.514869
[22]	valid_0's rmse: 0.512911
[23]	valid_0's rmse: 0.511025
[24]	valid_0's rmse: 0.509338
[25]	valid_0's rmse: 0.507771
[26]	valid_0's rm

In [23]:
X_test=pd.read_csv('Archivos de datos/test_values.csv')
X_test.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,300051,17,596,11307,3,20,7,6,t,r,...,0,0,0,0,0,0,0,0,0,0
1,99355,6,141,11987,2,25,13,5,t,r,...,1,0,0,0,0,0,0,0,0,0
2,890251,22,19,10044,2,5,4,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,745817,26,39,633,1,0,19,3,t,r,...,0,0,1,0,0,0,0,0,0,0
4,421793,17,289,7970,3,15,8,7,t,r,...,0,0,0,0,0,0,0,0,0,0


In [24]:
prediction=gbm.predict(X, num_iteration=gbm.best_iteration)
result=pd.DataFrame(prediction)

In [25]:
result['building_id']=X_test['building_id']
result.rename(columns={0:'damage_grade'},inplace=True)
result=result[['building_id','damage_grade']]
result.head()

,building_id,damage_grade
0,300051.0,2.314308
1,99355.0,2.534826
2,890251.0,2.645057
3,745817.0,2.037357
4,421793.0,2.493810


In [26]:
result.to_csv('Archivos de datos/resultado.csv',index=False)

![Submit](Imagenes/lightbm_submit.PNG)